In [ ]:
"""Combining Two Tables"""

select p.FirstName, p.LastName,
a.City, a.state
from person p
left join
address a 
on p.PersonId = a.PersonId

"""Second Highest Salary"""

select max(Salary) as SecondHighestSalary
from
(select Salary, 
Dense_Rank() over(order by Salary desc) as "Rnk"
from
Employee) as X
where X.Rnk = 2


"""Nth Highest Salary"""

CREATE FUNCTION getNthHighestSalary(N INT) RETURNS INT
BEGIN
  SET N=N-1;
    RETURN (
      # Write your MySQL query statement below.
      SELECT ifnull(
        (
            SELECT DISTINCT(salary)
            FROM Employee
            ORDER BY salary DESC
            LIMIT 1
            OFFSET N
        ), NULL
      )
  );
END


"""Swap Salary"""

update salary
set 
sex = Case sex
when 'm' then 'f'
else 'm'
end

"""Delete Duplicate Emails"""

Delete p1 from Person p1, person p2
where p1.Email = p2.Email and p1.Id > p2.Id


"""Department Top Three Salaries"""

select Department, Employee, Salary
from
(select d.Name as Department,
e.Name as Employee, dense_rank() 
over(partition by d.Name order by 
     e.Salary desc) as row_number,
e.Salary
from Employee e join department d
on e.DepartmentId = d.Id) as X
where X.row_number <=3


"""Employees Earning More Than Their Managers"""

select a.Name as "Employee" 
from Employee a
join Employee b
on a.ManagerId = b.Id
and a.Salary > b.Salary


"""Department Highest Salary"""

select d.Name as Department,
X.Name as Employee,
X.Salary
from Department d
join
(select * from (select Name, Salary, DepartmentId,
               dense_rank() over(partition by DepartmentId order by salary desc) as Rank
               from Employee) tmp where tmp.Rank = 1) X
on d.Id = X.DepartmentId
               

"""Trips and Users"""

select 
t.request_at as Day,
round(SUM(CASE WHEN (t.Status <> 'completed') then 1 else 0 end)/Count(t.Id), 2) as "Cancellation Rate"
from trips t 
join users u
on t.Client_Id = u.Users_Id
and u.Banned != 'Yes'
where t.Request_at between '2013-10-01' and '2013-10-03'
group by t.request_at


"""Human Traffic of Stadium"""

SELECT DISTINCT S1.*
FROM stadium S1
JOIN stadium S2
JOIN stadium S3
ON ((S1.id = S2.id - 1 AND S1.id = S3.id -2)
OR (S3.id = S1.id - 1 AND S3.id = S2.id -2)
OR (S3.id = S2.id - 1 AND S3.id = S1.id -2))
WHERE S1.people >= 100
AND S2.people >= 100
AND S3.people >= 100
ORDER BY S1.id;


"""Consecutive Numbers"""

select distinct l1.Num as ConsecutiveNums
from logs l1,
logs l2, 
logs l3
where l1.id = l2.id -1
and l2.id = l3.id - 1
and l1.num = l2.num
and l2.num = l3.num


"""Find Cumulative Salary of an Employee"""

select Id, Month, cumsum as salary
from
(select *, Sum(Salary) over(partition by Id order by Month ROWS 2 PRECEDING) as cumsum
from 
(select *, dense_rank() over(partition by Id order by Month desc) as rankmonth
 from Employee) a where rankmonth<>1) b
order by id, Month desc
-- group  by Month


"""Average Salary: Departments VS Company"""

select department_salary.pay_month, department_id, 
case
    when department_avg > company_avg then 'higher'
    when department_avg < company_avg then 'lower'
    else 'same'
end as comparison
from
(select department_id, avg(amount) as department_avg, date_format(pay_date, '%Y-%m') as pay_month
from salary join employee on salary.employee_id = employee.employee_id
group by department_id, pay_month) as department_salary
join
(select avg(amount) as company_avg,date_format(pay_date, '%Y-%m') as pay_month
from salary group by date_format(pay_date, '%Y-%m')) as company_salary
on department_salary.pay_month = company_salary.pay_month


